In [7]:
import os
import numpy as np
import pandas as pd
import IPython.display as ipd
import tgt
import math
import sklearn
import librosa
import scipy.io.wavfile
from numpy import unique
from numpy import where
from matplotlib import pyplot as plt
import librosa.display
from sklearn import metrics
%matplotlib inline

In [2]:
def readTextGrid(filename,word,pronunciation_vowel):
    intervals=[]
    
    try:
        text_grid= tgt.read_textgrid(filename)
    
    except Exception as FileNotFoundError:
        print("File does not exist")
   
    for tier in text_grid.tiers:
        
        try:
            if(tier.get_annotations_with_text(pattern=word, n=0, regex=False)):
                intervals.append(tier.get_annotations_with_text(pattern=word, n=0, regex=False)[0])
                interval=tier.get_annotations_with_text(pattern=word, n=0, regex=False)[0]
                start_time_word=interval.start_time
                end_time_word=interval.end_time

            elif(tier.get_annotations_with_text(pattern=pronunciation_vowel, n=0, regex=False)):
                intervals.append(tier.get_annotations_with_text(pattern=pronunciation_vowel, n=0, regex=False)[0])
                interval=tier.get_annotations_with_text(pattern=pronunciation_vowel, n=0, regex=False)[0]
                start_time_pronunciation_vowel=interval.start_time
                end_time_pronunciation_vowel=interval.end_time

        except Exception as e:
            print(e)
    return start_time_word,end_time_word,start_time_pronunciation_vowel,end_time_pronunciation_vowel


In [3]:
def audio_trimming(audio_file,start_time,end_time):
    y1,fs1=librosa.load(audio_file,sr=16000)
   # fs1, y1 = scipy.io.wavfile.read(audio_file)
    #fs1,y1
    print("start and end time is ",start_time,end_time)
    l1=np.array([[start_time,end_time]])
    l1=np.ceil(l1*fs1)
    newWavFileAsList = []
    for elem in l1:
        startRead = elem[0]
        endRead = elem[1]
       
        if startRead >= y1.shape[0]:
            startRead = y1.shape[0]-1
        if endRead >= y1.shape[0]:
            endRead = y1.shape[0]-1
        newWavFileAsList.extend(y1[int(startRead):int(endRead)])

    newWavFile = np.array(newWavFileAsList)
    print(newWavFile.shape)
   # scipy.io.wavfile.write('test5.wav', fs1, newWavFile)
    #print("file saved ")
    return newWavFile
    

In [4]:
def load_displayAudio(filename):
    y,sr=librosa.load(filename,sr=16000)
    print(y.shape)
    plt.figure(figsize=(14, 5))
    librosa.display.waveplot(y, sr=sr)

In [5]:
def toFeature(file_paths,base_path_textGrid,base_path_audio,word,pronunciation_vowel):
    features_word=[]
    features_vowel=[]
    filenames=[]
    SAMPLE_RATE=16000
    i=0

    for path in file_paths:
        filenames.append(path)
        
        try:
            print(i)
            i=i+1
        
            #X, sample_rate = librosa.load(os.path.join(base_path,path),sr=SAMPLE_RATE)
            
            textGrid_file_path=os.path.join(base_path_textGrid,path)+'.TextGrid'
            start_time_word,end_time_word,start_time_pronunciation_vowel,end_time_pronunciation_vowel=readTextGrid(textGrid_file_path,word,pronunciation_vowel)
            
            print(start_time_word,end_time_word,start_time_pronunciation_vowel,end_time_pronunciation_vowel)
            audio_file_path=os.path.join(base_path_audio,path)+'.wav'
            
            newWavFile_word=audio_trimming(audio_file_path,start_time_word,end_time_word)
            newWavFile_vowel=audio_trimming(audio_file_path,start_time_pronunciation_vowel,end_time_pronunciation_vowel)
            
            sample_rate = np.array(SAMPLE_RATE)
            mfccs_word = np.mean(librosa.feature.mfcc(y=newWavFile_word,sr=sample_rate,n_mfcc=13),axis=1)
            mfccs_vowel = np.mean(librosa.feature.mfcc(y=newWavFile_vowel,sr=sample_rate,n_mfcc=13),axis=1)
            
            features_word.append(mfccs_word)
            features_vowel.append(mfccs_vowel)
            
        except:
            print("Exception is caused because of the file ",path)
            features_word.append()
            features_vowel.append()
            
    df_features_word,df_features_vowel=make_dataframe(filenames,features_word,features_vowel)
    return df_features_word,df_features_vowel

In [6]:
def make_dataframe(filenames,features_word,features_vowel):
    
    df_word=pd.DataFrame({'feature':features_word})
    df_vowel=pd.DataFrame({'feature':features_vowel})
    
    
    
    df_word=df_word.feature.apply(pd.Series)
    df_vowel=df_vowel.feature.apply(pd.Series)
    
    
    df_word['file_name']=filenames
    df_vowel['file_name']=filenames
    
    return df_word,df_vowel
    

In [8]:
if __name__=="__main__":
    # three variables are needed here the two  basepaths and the dataframe path
    # as an additional feature we can definitely ask for the feature to be extracted
    base_path_textGrid="D://Himani-work/gsoc2020/dataset/ideology_extra200ms/textgrids_for_extra_200ms_MAUS/"
    base_path_audio="D://Himani-work/gsoc2020/dataset/ideology_extra200ms/with_extra_200ms"
    
    textGridFiles=list(os.listdir(base_path_textGrid))
    audioFiles=list(os.listdir(base_path_audio))
    all_audio_files=list(pd.DataFrame({'audioFile':audioFiles})['audioFile'].str.split(".wav",expand=True).iloc[:,0])
    all_textGrid_files=list(pd.DataFrame({'textGridFile':textGridFiles})['textGridFile'].str.split(".TextGrid",expand=True).iloc[:,0])
    print("total texGrid and audio files",len(all_audio_files),len(all_textGrid_files))
    
    df_ideology_200ms=pd.read_csv('D:/Himani-work/gsoc2020/dataset/spreadsheet_data/ideology_results_praat_formants_extracted_with_200ms_for_R.csv',sep='\t')
    all_correct_files=list(df_ideology_200ms['file'].str.split("\\",expand=True).iloc[:,-1].str.split(".wav",expand=True).iloc[:,0])
    print("total corrected files",len(all_correct_files))
    
    final_files=list(set(all_audio_files)&set(all_correct_files)&set(all_textGrid_files))
    print("final files",len(final_files))
    
    
    # Flow is like call the 
    word="ideology"
    pronunciation_vowel="aI"


    df_features_word,df_features_vowel=toFeature(final_files,base_path_textGrid,base_path_audio,word,pronunciation_vowel)

    df_features_word.to_csv('mfcc0_features_word.csv',index=False)
    df_features_vowel.to_csv('mfcc0.features.vowel.csv',index=False)
    
    
    
    
    


total texGrid and audio files 2792 2699
total corrected files 2696
final files 2696
0
0.0 0.98 0.0 0.16
start and end time is  0.0 0.98
(15679,)
start and end time is  0.0 0.16
(2560,)
1
0.0 1.01 0.0 0.39
start and end time is  0.0 1.01
(16159,)
start and end time is  0.0 0.39
(6240,)
2
0.0 0.86 0.0 0.07
start and end time is  0.0 0.86
(13759,)
start and end time is  0.0 0.07
(1120,)
3
0.19 0.98 0.19 0.38
start and end time is  0.19 0.98
(12639,)
start and end time is  0.19 0.38
(3040,)
4
0.0 0.97 0.0 0.34
start and end time is  0.0 0.97
(15520,)
start and end time is  0.0 0.34
(5440,)
5
0.0 1.15 0.0 0.43
start and end time is  0.0 1.15
(18399,)
start and end time is  0.0 0.43
(6880,)
6
0.0 1.06 0.0 0.52
start and end time is  0.0 1.06
(16959,)
start and end time is  0.0 0.52
(8320,)
7
0.0 0.83 0.0 0.32
start and end time is  0.0 0.83
(13279,)
start and end time is  0.0 0.32
(5120,)
8
0.23 1.06 0.23 0.44
start and end time is  0.23 1.06
(13279,)
start and end time is  0.23 0.44
(3360,)

0.1 0.98 0.1 0.35
start and end time is  0.1 0.98
(14079,)
start and end time is  0.1 0.35
(4000,)
81
0.0 0.85 0.0 0.34
start and end time is  0.0 0.85
(13599,)
start and end time is  0.0 0.34
(5440,)
82
0.06 1.01 0.06 0.4
start and end time is  0.06 1.01
(15199,)
start and end time is  0.06 0.4
(5440,)
83
0.0 0.77 0.0 0.11
start and end time is  0.0 0.77
(12319,)
start and end time is  0.0 0.11
(1760,)
84
0.0 0.96 0.0 0.34
start and end time is  0.0 0.96
(15359,)
start and end time is  0.0 0.34
(5440,)
85
0.03 1.13 0.03 0.22
start and end time is  0.03 1.13
(17600,)
start and end time is  0.03 0.22
(3040,)
86
0.0 0.83 0.0 0.36
start and end time is  0.0 0.83
(13279,)
start and end time is  0.0 0.36
(5760,)
87
0.04 0.85 0.04 0.19
start and end time is  0.04 0.85
(12959,)
start and end time is  0.04 0.19
(2400,)
88
0.03 0.98 0.03 0.34
start and end time is  0.03 0.98
(15199,)
start and end time is  0.03 0.34
(4960,)
89
0.0 0.86 0.0 0.33
start and end time is  0.0 0.86
(13759,)
start and

start and end time is  0.04 0.31
(4320,)
161
0.28 1.0 0.28 0.34
start and end time is  0.28 1.0
(11520,)
start and end time is  0.28 0.34
(960,)
162
0.16 0.9 0.16 0.37
start and end time is  0.16 0.9
(11839,)
start and end time is  0.16 0.37
(3360,)
163
0.0 0.96 0.0 0.36
start and end time is  0.0 0.96
(15359,)
start and end time is  0.0 0.36
(5760,)
164
0.07 0.6 0.07 0.11
start and end time is  0.07 0.6
(8479,)
start and end time is  0.07 0.11
(640,)
165
0.0 0.95 0.0 0.39
start and end time is  0.0 0.95
(15199,)
start and end time is  0.0 0.39
(6240,)
166
0.0 0.84 0.0 0.35
start and end time is  0.0 0.84
(13439,)
start and end time is  0.0 0.35
(5600,)
167
0.0 0.83 0.0 0.36
start and end time is  0.0 0.83
(13279,)
start and end time is  0.0 0.36
(5760,)
168
0.07 0.97 0.07 0.39
start and end time is  0.07 0.97
(14399,)
start and end time is  0.07 0.39
(5120,)
169
0.0 0.86 0.0 0.04
start and end time is  0.0 0.86
(13759,)
start and end time is  0.0 0.04
(640,)
170
0.0 0.6 0.0 0.13
start

0.12 0.78 0.12 0.36
start and end time is  0.12 0.78
(10559,)
start and end time is  0.12 0.36
(3840,)
241
0.0 0.54 0.0 0.1
start and end time is  0.0 0.54
(8639,)
start and end time is  0.0 0.1
(1600,)
242
0.13 1.02 0.13 0.38
start and end time is  0.13 1.02
(14239,)
start and end time is  0.13 0.38
(4000,)
243
0.0 0.78 0.0 0.34
start and end time is  0.0 0.78
(12479,)
start and end time is  0.0 0.34
(5440,)
244
0.11 0.92 0.11 0.37
start and end time is  0.11 0.92
(12959,)
start and end time is  0.11 0.37
(4160,)
245
0.0 0.85 0.0 0.17
start and end time is  0.0 0.85
(13599,)
start and end time is  0.0 0.17
(2720,)
246
0.11 0.87 0.11 0.2
start and end time is  0.11 0.87
(12159,)
start and end time is  0.11 0.2
(1440,)
247
0.12 1.01 0.12 0.4
start and end time is  0.12 1.01
(14239,)
start and end time is  0.12 0.4
(4480,)
248
0.0 0.87 0.0 0.38
start and end time is  0.0 0.87
(13919,)
start and end time is  0.0 0.38
(6080,)
249
0.06 0.58 0.06 0.15
start and end time is  0.06 0.58
(8320,)

322
0.01 0.63 0.01 0.18
start and end time is  0.01 0.63
(9919,)
start and end time is  0.01 0.18
(2720,)
323
0.02 0.91 0.02 0.37
start and end time is  0.02 0.91
(14239,)
start and end time is  0.02 0.37
(5600,)
324
0.0 0.89 0.0 0.12
start and end time is  0.0 0.89
(14239,)
start and end time is  0.0 0.12
(1920,)
325
0.11 0.85 0.11 0.31
start and end time is  0.11 0.85
(11839,)
start and end time is  0.11 0.31
(3200,)
326
0.0 1.02 0.0 0.41
start and end time is  0.0 1.02
(16319,)
start and end time is  0.0 0.41
(6560,)
327
0.0 0.85 0.0 0.34
start and end time is  0.0 0.85
(13599,)
start and end time is  0.0 0.34
(5440,)
328
0.0 1.08 0.0 0.36
start and end time is  0.0 1.08
(17279,)
start and end time is  0.0 0.36
(5760,)
329
0.0 1.06 0.0 0.18
start and end time is  0.0 1.06
(16960,)
start and end time is  0.0 0.18
(2880,)
330
0.11 0.78 0.11 0.25
start and end time is  0.11 0.78
(10719,)
start and end time is  0.11 0.25
(2240,)
331
0.02 0.54 0.02 0.13
start and end time is  0.02 0.54
(

...

2337
0.0 1.07 0.0 0.09
start and end time is  0.0 1.07
(17120,)
start and end time is  0.0 0.09
(1440,)
2338
0.0 1.03 0.0 0.38
start and end time is  0.0 1.03
(16479,)
start and end time is  0.0 0.38
(6080,)
2339
0.21 1.08 0.21 0.39
start and end time is  0.21 1.08
(13919,)
start and end time is  0.21 0.39
(2880,)
2340
0.0 0.71 0.0 0.19
start and end time is  0.0 0.71
(11359,)
start and end time is  0.0 0.19
(3040,)
2341
0.09 0.65 0.09 0.2
start and end time is  0.09 0.65
(8959,)
start and end time is  0.09 0.2
(1760,)
2342
0.0 0.77 0.0 0.07
start and end time is  0.0 0.77
(12319,)
start and end time is  0.0 0.07
(1120,)
2343
0.24 1.01 0.24 0.33
start and end time is  0.24 1.01
(12320,)
start and end time is  0.24 0.33
(1440,)
2344
0.0 1.02 0.0 0.39
start and end time is  0.0 1.02
(16319,)
start and end time is  0.0 0.39
(6240,)
2345
0.0 0.94 0.0 0.33
start and end time is  0.0 0.94
(15039,)
start and end time is  0.0 0.33
(5280,)
2346
0.2 1.02 0.2 0.34
start and end time is  0.2 1.02


0.0 1.14 0.0 0.15
start and end time is  0.0 1.14
(18240,)
start and end time is  0.0 0.15
(2400,)
2420
0.0 0.75 0.0 0.29
start and end time is  0.0 0.75
(11999,)
start and end time is  0.0 0.29
(4640,)
2421
0.0 1.09 0.0 0.4
start and end time is  0.0 1.09
(17440,)
start and end time is  0.0 0.4
(6400,)
2422
0.0 1.03 0.0 0.39
start and end time is  0.0 1.03
(16479,)
start and end time is  0.0 0.39
(6240,)
2423
0.04 1.35 0.04 0.44
start and end time is  0.04 1.35
(20959,)
start and end time is  0.04 0.44
(6400,)
2424
0.14 0.93 0.14 0.19
start and end time is  0.14 0.93
(12639,)
start and end time is  0.14 0.19
(800,)
2425
0.25 1.07 0.25 0.39
start and end time is  0.25 1.07
(13119,)
start and end time is  0.25 0.39
(2240,)
2426
0.09 0.83 0.09 0.16
start and end time is  0.09 0.83
(11839,)
start and end time is  0.09 0.16
(1120,)
2427
0.0 0.99 0.0 0.35
start and end time is  0.0 0.99
(15839,)
start and end time is  0.0 0.35
(5600,)
2428
0.0 1.1 0.0 0.4
start and end time is  0.0 1.1
(175

0.17 1.12 0.17 0.42
start and end time is  0.17 1.12
(15199,)
start and end time is  0.17 0.42
(4000,)
2502
0.0 0.8 0.0 0.04
start and end time is  0.0 0.8
(12799,)
start and end time is  0.0 0.04
(640,)
2503
0.0 0.94 0.0 0.33
start and end time is  0.0 0.94
(15040,)
start and end time is  0.0 0.33
(5280,)
2504
0.08 0.9 0.08 0.12
start and end time is  0.08 0.9
(13119,)
start and end time is  0.08 0.12
(640,)
2505
0.21 1.11 0.21 0.36
start and end time is  0.21 1.11
(14399,)
start and end time is  0.21 0.36
(2400,)
2506
0.0 0.87 0.0 0.37
start and end time is  0.0 0.87
(13919,)
start and end time is  0.0 0.37
(5920,)
2507
0.0 0.81 0.0 0.05
start and end time is  0.0 0.81
(12959,)
start and end time is  0.0 0.05
(800,)
2508
0.0 0.8 0.0 0.05
start and end time is  0.0 0.8
(12799,)
start and end time is  0.0 0.05
(800,)
2509
0.17 0.85 0.17 0.41
start and end time is  0.17 0.85
(10879,)
start and end time is  0.17 0.41
(3840,)
2510
0.0 0.74 0.0 0.04
start and end time is  0.0 0.74
(11839,)

0.18 0.89 0.18 0.33
start and end time is  0.18 0.89
(11360,)
start and end time is  0.18 0.33
(2400,)
2581
0.05 0.77 0.05 0.3
start and end time is  0.05 0.77
(11519,)
start and end time is  0.05 0.3
(4000,)
2582
0.0 1.28 0.0 0.49
start and end time is  0.0 1.28
(20479,)
start and end time is  0.0 0.49
(7840,)
2583
0.0 0.79 0.0 0.07
start and end time is  0.0 0.79
(12639,)
start and end time is  0.0 0.07
(1120,)
2584
0.0 0.86 0.0 0.4
start and end time is  0.0 0.86
(13759,)
start and end time is  0.0 0.4
(6400,)
2585
0.0 1.0 0.0 0.04
start and end time is  0.0 1.0
(15999,)
start and end time is  0.0 0.04
(640,)
2586
0.0 1.07 0.0 0.35
start and end time is  0.0 1.07
(17119,)
start and end time is  0.0 0.35
(5600,)
2587
0.18 0.84 0.18 0.31
start and end time is  0.18 0.84
(10559,)
start and end time is  0.18 0.31
(2080,)
2588
0.13 1.04 0.13 0.37
start and end time is  0.13 1.04
(14559,)
start and end time is  0.13 0.37
(3840,)
2589
0.02 0.62 0.02 0.06
start and end time is  0.02 0.62
(9

0.02 0.83 0.02 0.34
start and end time is  0.02 0.83
(12959,)
start and end time is  0.02 0.34
(5120,)
2659
0.0 1.13 0.0 0.09
start and end time is  0.0 1.13
(18079,)
start and end time is  0.0 0.09
(1440,)
2660
0.0 0.77 0.0 0.1
start and end time is  0.0 0.77
(12319,)
start and end time is  0.0 0.1
(1600,)
2661
0.01 1.05 0.01 0.46
start and end time is  0.01 1.05
(16639,)
start and end time is  0.01 0.46
(7200,)
2662
0.0 0.899937 0.0 0.04
start and end time is  0.0 0.899937
(14398,)
start and end time is  0.0 0.04
(640,)
2663
0.0 1.13 0.0 0.37
start and end time is  0.0 1.13
(18079,)
start and end time is  0.0 0.37
(5920,)
2664
0.0 0.95 0.0 0.35
start and end time is  0.0 0.95
(15199,)
start and end time is  0.0 0.35
(5600,)
2665
0.03 1.01 0.03 0.15
start and end time is  0.03 1.01
(15679,)
start and end time is  0.03 0.15
(1920,)
2666
0.22 1.1 0.22 0.4
start and end time is  0.22 1.1
(14079,)
start and end time is  0.22 0.4
(2880,)
2667
0.0 0.56 0.0 0.04
start and end time is  0.0 0.

In [9]:
len(df_features_vowel),len(df_features_word)

(2696, 2696)

In [10]:
final_files[-1]

'2016-02-25_0200_US_HLN_HLN_Dr_Drew_clip_2648_ai__399.97-400.48__0.01'

In [ ]:
word="ideology"
pronunciation_vowel="aI"


df_features_word,df_features_vowel=toFeature(final_files[0:2],base_path_textGrid,base_path_audio,word,pronunciation_vowel)


In [ ]:
df_features_vowel.head(2)

In [ ]:
df_features_word.head(2)

In [ ]:
word="ideology"
pronunciation_vowel="aI"
filenames,features_word,features_vowel=toFeature(final_files[0:2],base_path_textGrid,base_path_audio,word,pronunciation_vowel)

In [ ]:
df_features=pd.DataFrame({'feature':features_word})
df_features=df_features.feature.apply(pd.Series)
df_features['file_name']=filenames

In [ ]:
df_features


In [ ]:
df_features=pd.DataFrame({'file_name':filenames,'feature':features_word})
print(df_features.shape)
df_features.head(2)

In [ ]:
df_features=pd.DataFrame({'feature':features_word})
print(df_features.shape)
df_features.head(2)

In [ ]:
df_features['feature'].values.tolist()

In [ ]:
df_final_Audio=pd.DataFrame(df_features['feature'].values.tolist())
df_final_Audio.head(2)

In [ ]:
def mfccFeature(filename):
    #1. get the extracted start time and end time 
    #2. get the trimmed audio 
    #3. get the feature 

In [ ]:
base_path_textGrid="D://Himani-work/gsoc2020/dataset/ideology_extra200ms/textgrids_for_extra_200ms_MAUS/"
base_path_audio="D://Himani-work/gsoc2020/dataset/ideology_extra200ms/with_extra_200ms"
textGridFiles=list(os.listdir(base_path_textGrid))
audioFiles=list(os.listdir(base_path_audio))
len(textGridFiles),len(audioFiles)

In [ ]:
df_ideology_200ms=pd.read_csv('D:/Himani-work/gsoc2020/dataset/spreadsheet_data/ideology_results_praat_formants_extracted_with_200ms_for_R.csv',sep='\t')
print(df_ideology_200ms.shape)
df_ideology_200ms.head(2)

In [ ]:
all_correct_files=list(df_ideology_200ms['file'].str.split("\\",expand=True).iloc[:,-1].str.split(".wav",expand=True).iloc[:,0])
len(all_correct_files)

In [ ]:
all_audio_files=list(pd.DataFrame({'audioFile':audioFiles})['audioFile'].str.split(".wav",expand=True).iloc[:,0])
all_textGrid_files=list(pd.DataFrame({'textGridFile':textGridFiles})['textGridFile'].str.split(".TextGrid",expand=True).iloc[:,0])
len(all_audio_files),len(all_textGrid_files)

In [ ]:
all_audio_files[56],all_textGrid_files[56],all_correct_files[56]

In [ ]:
len(set(all_audio_files)&set(all_correct_files)&set(all_textGrid_files))

In [ ]:
final_files=list(set(all_audio_files)&set(all_correct_files)&set(all_textGrid_files))
len(final_files)

In [ ]:
start_time_word,end_time_word,start_time_pronunciation_vowel,end_time_pronunciation_vowel=readTextGrid(os.path.join(base_path_textGrid,final_files[0])+'.TextGrid','ideology','aI')
start_time_word,end_time_word,start_time_pronunciation_vowel,end_time_pronunciation_vowel

In [ ]:
audio_file=os.path.join(base_path_audio,final_files[0])+'.wav'
audio_file

In [ ]:
newWavFile=audio_trimming(audio_file,start_time_pronunciation_vowel,end_time_pronunciation_vowel)
load_displayAudio(audio_file)
plt.figure(figsize=(14, 5))
plt.title("The vowel sound")
librosa.display.waveplot(newWavFile.astype(np.float32), sr=16000)

In [ ]:
mfccs = np.mean(librosa.feature.mfcc(y=newWavFile,sr=16000,n_mfcc=13),axis=1)

In [ ]:
mfccs.shape

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(3,1,1)
mfcc=librosa.feature.mfcc(y=newWavFile,sr=16000,n_mfcc=13)
#plt.title("MFCC is for sound %s " %sound)
librosa.display.specshow(mfcc)
plt.ylabel('MFCC')
plt.colorbar()

In [ ]:
final_files[0]

In [ ]:
#lets test run the textGrid to see the correct
check=[]
for file in final_files:
    #print(os.path.join(base_path_textGrid,file))
    s,e,ss,ee=readTextGrid(os.path.join(base_path_textGrid,file)+'.TextGrid','ideology','aI')
    check.append(s)
len(check)
    